# InvarLock Quickstart (CPU, Colab)

**Purpose:** Install InvarLock, run a small certification, verify the resulting certificate, and export HTML.
**Runtime:** ~5 minutes (CPU; longer on slow machines)
**Requires:** `invarlock[hf]` and network for first downloads

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/invarlock/invarlock/blob/main/notebooks/invarlock_quickstart_cpu.ipynb)

This notebook installs InvarLock with the Hugging Face extra (`invarlock[hf]`), runs a small certification on GPT-2 small, and verifies the resulting certificate.

It follows the documented "hero path" in two flavors:

1. Install `invarlock[hf]` (HF/torch stack; core `invarlock` install remains torch-free)
2. Run `invarlock certify` on a small causal LM (dev + tiny-relax smoke)
3. Run `invarlock verify` on the generated `evaluation.cert.json`

The final section also shows example `certify` commands for CI/Release with calibrated presets, for when you want to work with configs or this repo.


In [ ]:
!pip -q install "invarlock[hf]"
import sys

import invarlock

print("Python:", sys.version.split()[0])
print("InvarLock:", invarlock.__version__)
!invarlock doctor --json || true

> Note: On Colab you may see pip dependency conflict warnings about preinstalled packages (for example, `gcsfs` or `google-adk`). These come from the Colab environment, not InvarLock. If `invarlock doctor --json` reports zero errors and zero warnings, your install is healthy.


In [ ]:
%%bash
export INVARLOCK_ALLOW_NETWORK=1 INVARLOCK_DEDUP_TEXTS=1 INVARLOCK_TINY_RELAX=1 TRANSFORMERS_NO_TORCHVISION=1 TOKENIZERS_PARALLELISM=false
invarlock certify \
  --baseline gpt2 \
  --subject  gpt2 \
  --adapter auto \
  --profile dev \
  --cert-out reports/cert


> Note: Colab may print TensorFlow/CUDA log messages or occasional CUDA out-of-memory warnings when loading models. These are environment-level messages. For this quickstart, the run is considered successful if you see `Run completed successfully!` and the certificate summary shows `PASS`.


In [ ]:
%%bash
invarlock verify --json reports/cert/evaluation.cert.json
invarlock report html -i reports/cert/evaluation.cert.json -o reports/cert/evaluation.html
echo "Open: reports/cert/evaluation.html"


## CI/Release examples (for reference)

Once you're comfortable with the quick dev smoke above, you can run calibrated CI/Release certifications on GPT-2 small or other supported models.

For example, after cloning this repo or writing an inline preset equivalent to `configs/presets/causal_lm/wikitext2_512.yaml`:

```bash
INVARLOCK_ALLOW_NETWORK=1 INVARLOCK_DEDUP_TEXTS=1 \
invarlock certify \
  --baseline gpt2 \
  --subject  gpt2 \
  --adapter  auto \
  --profile  release \
  --preset   configs/presets/causal_lm/wikitext2_512.yaml \
  --out      runs/release_gpt2 \
  --cert-out reports/cert_release_gpt2
```

Similar flows work for BERT (masked LM); see the docs for calibrated model families.
